In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import MiniBatchKMeans
import random
from sklearn.neighbors import NearestNeighbors
import numpy as np
import collections
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
import os;
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from matplotlib import pyplot

In [ ]:
path = "/Users/emily/Desktop/Research/oversampling_python"
dirs = os.listdir(path+"/data/pima-5-fold")
os.chdir(path+"/data/pima-5-fold")
# 到時候 path 這邊也要 loop 不同資料集的資料夾
# 输出所有文件和文件夹
train = []
test = []
allfile = []
for file in dirs:
    if("xlsx" in file):
        allfile.append(file)
        if("tra" in file ):
            train.append(file)
        elif("tst" in file):
            test.append(file)
allfile

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
import statistics
from sklearn import preprocessing
from sklearn import metrics
os.chdir("/Users/emily/Desktop/Research/oversampling_python/data/pima-5-fold")
accuracies=[]
oversampleRCSMOTE = [] # 全部 5 個 train data 所訓練出來的 oversample data
for i in train:
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]

    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
    Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
    Range = range_value(3,finaldata,output,data)
    P_max = range_value(3,finaldata,output,data)[0]
    Range = range_value(3,finaldata,output,data)[2]
    alldata = data.T
    classCount = classprocess(output)
    
    tempover = []
    for i in range(len(classCount)):
        if(int(classCount[i][1]) > 0):
            over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])
            tempover.extend(over)
            
            length = alldata.shape[1]
            for j in range(len(over)):
                alldata[length+j] = over[j]
    X = alldata.T.iloc[:,:alldata.shape[0]-1]
    y = alldata.T.iloc[:,alldata.shape[0]-1]
    clf = DecisionTreeClassifier()
    clf = clf.fit(X, y) # 訓練 model
    oversampleRCSMOTE.append(tempover)
    """ 畫樹
    dot_data = tree.export_graphviz(clf)
    fig = pyplot.figure(figsize=(25,20))
    _ = tree.plot_tree(clf)
    tree.plot_tree(clf) 
    """
    test_file = pd.read_excel(test[i],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    """
    le = preprocessing.LabelEncoder()
    for i in range(test_data.shape[1]):
        test_data.iloc[:,i] = le.fit_transform(test_data.iloc[:,i]) 
    """
    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)


In [ ]:
# SMOTE
from imblearn.over_sampling import SMOTE
overSMOTE = []
alloverSMOTE = []
for i in train:
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]
    tempover = []
    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    
    #tempover.extend([X_smote,y_smote])

    overSMOTE.append(alloverSMOTE)
    print(len(overSMOTE),"len")
    alloverSMOTE = [] 
print(len(overSMOTE[0]))
np.array(overSMOTE).shape



In [ ]:
# ranodom half SMOTE 
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = np.array(data.iloc[:,l]);
    classCount = classprocess(output)
    finaldata = np.array(data.iloc[:,:l])
    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    overSMOTE.append(alloverSMOTE)
    alloverSMOTE =[]
    
    for i in range(len(classCount)):
        count = math.floor(int(classCount[i][1])/2); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []

    
    for index in randomIndex:
       
        randomtemp.append(overSMOTE[ii][index])
        
    randomSMOTE.append(randomtemp)
    print(ii," ",len(randomtemp))
    print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
np.array(randomSMOTE).shape

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
# 產生 random RCSMOTE

randomRCSMOTE = []
for ii,i in enumerate(train):
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]

    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
    Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
    Range = range_value(3,finaldata,output,data)
    P_max = range_value(3,finaldata,output,data)[0]
    Range = range_value(3,finaldata,output,data)[2]
    alldata = data.T
    classCount = classprocess(output)
    randomIndex = []
    tempover = []
    for j in range(len(classCount)):
        if(int(classCount[j][1]) > 0):
            over = Populate(int(classCount[j][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[j][0])
            tempover.append(over)
            #length = alldata.shape[1]
    oversampleRCSMOTE.append(tempover)
 
    
    for a in range(len(classCount)): # 產生 random index
        count = math.ceil(int(classCount[a][1])/2); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(0,len(oversampleRCSMOTE[ii])-1) for _ in range(count)]) 
    
    randomtemp = []
    #print("randomindex",len(randomIndex))
    
    for index in randomIndex:
        randomtemp.append(oversampleRCSMOTE[ii][index])
    randomRCSMOTE.append(randomtemp)
    #print("actual",len(randomSMOTE[ii]))
 

    


In [ ]:
# random SMOTE + RCSMOTE 各一半的結果 
allRandomHalf = []
temp = []
for i in range(len(randomRCSMOTE)):
    temp = randomRCSMOTE[i] + randomSMOTE[i]
    allRandomHalf.append(temp)


In [ ]:
# 跟原始資料合併
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    classCount = classprocess(output)
    data = data.T
    
    
    for i in range(len(allRandomHalf[index])): # 185
        #for j in range(len(allRandomHalf[index][0])): #9
            datalength = data.shape[1]
            data[datalength+i+1] = allRandomHalf[index][j]
    #print(data)  
        #print(datalength)
        #print(allRandomHalf[index][i],"sl;d")
    mergeRandom = data.T
    l = mergeRandom.shape[1]-1
    output = np.array(mergeRandom.iloc[:,l]);
    finaldata = np.array(mergeRandom.iloc[:,:l])
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)
"""
    for i in range(len(classCount)):
    #print(int(classCount[i][1]))
        if(int(classCount[i][1]) > 0):
            #over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])
            length = alldata.shape[1]
            for j in range(len(over)):
                alldata[length+j] = over[j]
"""
#len(mergeRandom[0][0])


不同參數的合併 random

RCSMOTE : SMOTE = 7:3

In [ ]:
# 產生 random RCSMOTE 取 7 成

randomRCSMOTE = []
for ii,i in enumerate(train):
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]

    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
    Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
    Range = range_value(3,finaldata,output,data)
    P_max = range_value(3,finaldata,output,data)[0]
    Range = range_value(3,finaldata,output,data)[2]
    alldata = data.T
    classCount = classprocess(output)
    randomIndex = []
    tempover = []
    for j in range(len(classCount)):
        if(int(classCount[j][1]) > 0):
            over = Populate(int(classCount[j][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[j][0])
            tempover.append(over)
            #length = alldata.shape[1]
    oversampleRCSMOTE.append(tempover)
 
    
    for a in range(len(classCount)): # 產生 random index
        count = math.ceil(int(classCount[a][1])*0.7); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(0,len(oversampleRCSMOTE[ii])-1) for _ in range(count)]) 
    
    randomtemp = []
    #print("randomindex",len(randomIndex))
    
    for index in randomIndex:
        randomtemp.append(oversampleRCSMOTE[ii][index])
    randomRCSMOTE.append(randomtemp)
    #print("actual",len(randomSMOTE[ii]))
 

    


In [ ]:
# random SMOTE 取 3 成
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = np.array(data.iloc[:,l]);
    classCount = classprocess(output)
    finaldata = np.array(data.iloc[:,:l])
    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    overSMOTE.append(alloverSMOTE)
    alloverSMOTE =[]
    
    for i in range(len(classCount)):
        count = math.floor(int(classCount[i][1])*0.3); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []

    
    for index in randomIndex:
       
        randomtemp.append(overSMOTE[ii][index])
        
    randomSMOTE.append(randomtemp)
    print(ii," ",len(randomtemp))
    print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
np.array(randomSMOTE).shape

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
# random SMOTE : RCSMOTE = 3:7 的結果 
allRandomHalf = []
temp = []
for i in range(len(randomRCSMOTE)):
    temp = randomRCSMOTE[i] + randomSMOTE[i]
    allRandomHalf.append(temp)

In [ ]:
# 跟原始資料合併
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    classCount = classprocess(output)
    data = data.T
    
    
    for i in range(len(allRandomHalf[index])): # 185
        #for j in range(len(allRandomHalf[index][0])): #9
            datalength = data.shape[1]
            data[datalength+i+1] = allRandomHalf[index][j]
    #print(data)  
        #print(datalength)
        #print(allRandomHalf[index][i],"sl;d")
    mergeRandom = data.T
    l = mergeRandom.shape[1]-1
    output = np.array(mergeRandom.iloc[:,l]);
    finaldata = np.array(mergeRandom.iloc[:,:l])
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)
"""
    for i in range(len(classCount)):
    #print(int(classCount[i][1]))
        if(int(classCount[i][1]) > 0):
            #over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])
            length = alldata.shape[1]
            for j in range(len(over)):
                alldata[length+j] = over[j]
"""
#len(mergeRandom[0][0])


SMOTE : RCSMOTE = 7 : 3

In [ ]:
# 產生 random RCSMOTE 取 3 成

randomRCSMOTE = []
for ii,i in enumerate(train):
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]

    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
    Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
    Range = range_value(3,finaldata,output,data)
    P_max = range_value(3,finaldata,output,data)[0]
    Range = range_value(3,finaldata,output,data)[2]
    alldata = data.T
    classCount = classprocess(output)
    randomIndex = []
    tempover = []
    for j in range(len(classCount)):
        if(int(classCount[j][1]) > 0):
            over = Populate(int(classCount[j][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[j][0])
            tempover.append(over)
            #length = alldata.shape[1]
    oversampleRCSMOTE.append(tempover)
 
    
    for a in range(len(classCount)): # 產生 random index
        count = math.ceil(int(classCount[a][1])*0.3); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(0,len(oversampleRCSMOTE[ii])-1) for _ in range(count)]) 
    
    randomtemp = []
    #print("randomindex",len(randomIndex))
    
    for index in randomIndex:
        randomtemp.append(oversampleRCSMOTE[ii][index])
    randomRCSMOTE.append(randomtemp)
    #print("actual",len(randomSMOTE[ii]))
 

    


In [ ]:
# random SMOTE 取 7 成
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = np.array(data.iloc[:,l]);
    classCount = classprocess(output)
    finaldata = np.array(data.iloc[:,:l])
    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    overSMOTE.append(alloverSMOTE)
    alloverSMOTE =[]
    
    for i in range(len(classCount)):
        count = math.floor(int(classCount[i][1])*0.7); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []

    
    for index in randomIndex:
       
        randomtemp.append(overSMOTE[ii][index])
        
    randomSMOTE.append(randomtemp)
    print(ii," ",len(randomtemp))
    print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
np.array(randomSMOTE).shape

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
# random SMOTE : RCSMOTE = 3:7 的結果 
allRandomHalf = []
temp = []
for i in range(len(randomRCSMOTE)):
    temp = randomRCSMOTE[i] + randomSMOTE[i]
    allRandomHalf.append(temp)

In [ ]:
# 跟原始資料合併
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    classCount = classprocess(output)
    data = data.T
    
    
    for i in range(len(allRandomHalf[index])): # 185
        #for j in range(len(allRandomHalf[index][0])): #9
            datalength = data.shape[1]
            data[datalength+i+1] = allRandomHalf[index][j]
    #print(data)  
        #print(datalength)
        #print(allRandomHalf[index][i],"sl;d")
    mergeRandom = data.T
    l = mergeRandom.shape[1]-1
    output = np.array(mergeRandom.iloc[:,l]);
    finaldata = np.array(mergeRandom.iloc[:,:l])
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)
"""
    for i in range(len(classCount)):
    #print(int(classCount[i][1]))
        if(int(classCount[i][1]) > 0):
            #over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])
            length = alldata.shape[1]
            for j in range(len(over)):
                alldata[length+j] = over[j]
"""
#len(mergeRandom[0][0])


群中心正式


In [877]:
# RCSMOTE 群中心 取 一半 50%
from sklearn.cluster import KMeans  
randomRCSMOTE = []
centerRCSMOTE = []
for ii,i in enumerate(train):
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]

    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
    Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
    Range = range_value(3,finaldata,output,data)
    P_max = range_value(3,finaldata,output,data)[0]
    Range = range_value(3,finaldata,output,data)[2]
    alldata = data.T
    classCount = classprocess(output)
    randomIndex = []
    tempover = []
    for j in range(len(classCount)):
        if(int(classCount[j][1]) > 0):
            over = Populate(int(classCount[j][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[j][0])
            tempover.append(over)
            #length = alldata.shape[1]
    oversampleRCSMOTE.append(tempover)
    countfor = math.ceil(len(oversampleRCSMOTE[ii])*0.3)
    kmeans = KMeans(n_clusters=1)
    dtemp = pd.DataFrame(oversampleRCSMOTE[ii])
    X = dtemp.iloc[:,:dtemp.shape[1]-1]
    kmeans.fit(X)
    y_kmeans = kmeans.predict(X)
    centers = kmeans.cluster_centers_
    distance = []
    X = X.astype('float64')
    centers = centers.astype('float64')
    tempindata = {}
    for i in range(X.shape[0]-1): #129 列
        distance = []
        temp = 0;
        for j in range(X.shape[1]-1):#9 行
            temp = pow((centers[0][j]-X.iloc[i][j]),2)  
            tempindata[i] = temp
    
    distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
        #print(sorted(k.items(), key=lambda x:x[1]))
        #print(tempindata)
        #distance.append(tempindata)
        #print(np.array(distance).shape)
    #distancearr = np.array([distancesortemp.items,)
    centerRCSMOTE.append(distancesortemp[:countfor])
    #print(distancesortemp[:countfor])
    
    #distancesort.append(distancesortemp)
centerRCSMOTE[0]

noisy 231 
border 191 
safe 193 
all 615
191
noisy 231 
border 191 
safe 193 
all 615
191
noisy 231 
border 191 
safe 193 
all 615
noisy 231 
border 191 
safe 193 
all 615
noisy 231 
border 191 
safe 193 
all 615
215
400
noisy 241 
border 161 
safe 213 
all 615
161
noisy 241 
border 161 
safe 213 
all 615
161
noisy 241 
border 161 
safe 213 
all 615
noisy 241 
border 161 
safe 213 
all 615
noisy 241 
border 161 
safe 213 
all 615
215
400
noisy 239 
border 172 
safe 203 
all 614
172
noisy 239 
border 172 
safe 203 
all 614
172
noisy 239 
border 172 
safe 203 
all 614
noisy 239 
border 172 
safe 203 
all 614
noisy 239 
border 172 
safe 203 
all 614
214
400
noisy 238 
border 181 
safe 195 
all 614
181
noisy 238 
border 181 
safe 195 
all 614
181
noisy 238 
border 181 
safe 195 
all 614
noisy 238 
border 181 
safe 195 
all 614
noisy 238 
border 181 
safe 195 
all 614
214
400
noisy 225 
border 183 
safe 206 
all 614
183
noisy 225 
border 183 
safe 206 
all 614
183
noisy 225 
border 183 
saf

[(72, 0.13520767671410652),
 (74, 0.1881293571827031),
 (85, 0.7441252119817483),
 (105, 5.011220890354801),
 (154, 7.8829047482268715),
 (60, 9.189292188689743),
 (151, 9.259300824418853),
 (181, 10.362224430827212),
 (161, 12.640067352120113),
 (158, 15.912326580083768),
 (23, 16.259322015210323),
 (94, 17.547723832079022),
 (34, 17.617737324917325),
 (3, 17.702194372323184),
 (104, 18.590976757782094),
 (12, 19.77488743723588),
 (31, 21.641672626425915),
 (170, 21.91108217617063),
 (180, 24.17743196577719),
 (26, 26.275571420739016),
 (96, 26.608508787397277),
 (176, 31.489621397048353),
 (115, 34.90210212443601),
 (125, 36.16077856395299),
 (122, 36.4446906582301),
 (79, 41.77443076410595),
 (112, 44.38227711329824),
 (152, 44.80040198248739),
 (86, 47.04622736690066),
 (102, 52.05443523568941),
 (51, 54.652952824113235),
 (70, 56.584505048618276),
 (136, 60.270449643852025),
 (166, 60.9512278598697),
 (163, 65.78700304330995),
 (100, 66.76230959937779),
 (150, 67.52545697673258),


In [878]:
# SMOTE 群中心 一半
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []
centerSMOTE = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = np.array(data.iloc[:,l]);
    classCount = classprocess(output)
    finaldata = np.array(data.iloc[:,:l])
    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    overSMOTE.append(alloverSMOTE)
    alloverSMOTE =[]
    
    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.7); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overSMOTE[ii])
            X = dtemp.iloc[:,:dtemp.shape[1]-1]
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            for i in range(X.shape[0]-1): #129 列
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((centers[0][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
    
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
        #print(sorted(k.items(), key=lambda x:x[1]))
        #print(tempindata)
        #distance.append(tempindata)
        #print(np.array(distance).shape)
    #distancearr = np.array([distancesortemp.items,)
            centerSMOTE.append(distancesortemp[:countfor])
    #print(distancesortemp[:countfor])
    
    #distancesort.append(distancesortemp)
print(len(centerSMOTE[0]))

215
400
215
400
214
400
214
400
214
400
129


In [ ]:
# centerSOMTE centerRCSMOTE
# 原先 data overSMOTE  oversampleRCSMOTE

In [879]:
# centerSMOTEvalue  紀錄實際在群中心的 SMOTE 一半
centerSMOTEvalue =[]
for i in range(len(centerSMOTE)):
    alltemp = []
    for j in range(len(centerSMOTE[i])):
        indexSMOTE = centerSMOTE[i][j][0]
        #tempSMOTE = list(overSMOTE[i][indexSMOTE])
        alltemp.append(list(overSMOTE[i][indexSMOTE]))
    centerSMOTEvalue.append(alltemp)
  
len(centerSMOTEvalue[4])

130

In [880]:
# centerRCSMOTEvalue  紀錄實際在群中心的 RCSMOTE 一半

centerRCSMOTEvalue =[]
for i in range(len(centerRCSMOTE)):
    alltemp = []
    for j in range(len(centerRCSMOTE[i])):
        #indexSMOTE = centerSMOTE[i][j][0]
        indexRCSMOTE = centerRCSMOTE[i][j][0]
       
        #tempSMOTE = list(overSMOTE[i][indexSMOTE])
        alltemp.append(oversampleRCSMOTE[i][indexRCSMOTE])
    centerRCSMOTEvalue.append(alltemp)
        #print(type(tempSMOTE))
        #temp = tempSMOTE + tempRCSMOTE
        #alltemp.append(temp)
    #allCenterHalf.append(alltemp)
    


In [881]:
# 合併 center RCSMOTE + SMOTE 各 50 
allCenterHalf = []
temp = []
for i in range(len(centerRCSMOTE)):
    temp = centerRCSMOTEvalue[i] + centerSMOTEvalue[i]
    allCenterHalf.append(temp)
len(allCenterHalf[4])

186

In [882]:
# 測試
# 跟原始資料合併
mergeCenter = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    classCount = classprocess(output)
    data = data.T
    
    
    for i in range(len(allCenterHalf[index])): # 185
        #for j in range(len(allRandomHalf[index][0])): #9
            datalength = data.shape[1]
            data[datalength+i+1] = allCenterHalf[index][j]
    #print(data)  
        #print(datalength)
        #print(allRandomHalf[index][i],"sl;d")
    mergeCenter = data.T
    l = mergeCenter.shape[1]-1
    output = np.array(mergeCenter.iloc[:,l]);
    finaldata = np.array(mergeCenter.iloc[:,:l])
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)
"""
    for i in range(len(classCount)):
    #print(int(classCount[i][1]))
        if(int(classCount[i][1]) > 0):
            #over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])
            length = alldata.shape[1]
            for j in range(len(over)):
                alldata[length+j] = over[j]
"""
#len(mergeRandom[0][0])



215
400
215
400
214
400
214
400
214
400
0.8929971988795519


'\n    for i in range(len(classCount)):\n    #print(int(classCount[i][1]))\n        if(int(classCount[i][1]) > 0):\n            #over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])\n            length = alldata.shape[1]\n            for j in range(len(over)):\n                alldata[length+j] = over[j]\n'

In [ ]:
# 找群中心
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1)
X = dtemp.iloc[:,:dtemp.shape[1]-1]
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
centers = kmeans.cluster_centers_
distance = []
X = X.astype('float64')
centers = centers.astype('float64')
for i in range(X.shape[0]-1): #129 列
    temp = 0;
    for j in range(X.shape[1]-1):#9 行
        temp = pow((centers[0][j]-X.iloc[i][j]),2)
        #print(centers[0][j])
        #print(centers[0][j]-X.iloc[i][j])
    distance.append(temp)
distancesort = sorted(distance)
distancesort

群中心測試

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
#import seaborn as sns; 
#sns.set()
import numpy as np
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

#plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
#plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

In [ ]:
dtemp = pd.DataFrame(randomSMOTE[0])
X = dtemp.iloc[:,:dtemp.shape[1]-1]
X

In [ ]:
type(X.iloc[0,:])
w = np.array(X.iloc[0,:])
type(w)

In [ ]:
X.shape[0]

In [ ]:
# 找群中心
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1)
X = dtemp.iloc[:,:dtemp.shape[1]-1]
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
centers = kmeans.cluster_centers_
distance = []
X = X.astype('float64')
centers = centers.astype('float64')
for i in range(X.shape[0]-1): #129 列
    temp = 0;
    for j in range(X.shape[1]-1):#9 行
        temp = pow((centers[0][j]-X.iloc[i][j]),2)
        #print(centers[0][j])
        #print(centers[0][j]-X.iloc[i][j])
    distance.append(temp)
distancesort = sorted(distance)
distancesort

In [ ]:
distance

In [ ]:
 neighbors=NearestNeighbors(n_neighbors=10).fit(X)
 temp = neighbors.kneighbors(X,return_distance=False)
 print(centers)
index = "no"
"""
for i in range(len(centers[0])):
    for j in X
"""
"""
for i in range(X.shape[0]):
    w = np.array(X.iloc[i,:])
    print(w)
    if(centers == w.any()):
        index = i;
        print("yes",index)

"""


In [ ]:
iris.values

In [ ]:
from sklearn.cluster import KMeans
from sklearn import cluster, datasets, metrics
# 讀入鳶尾花資料

iris = datasets.load_iris()
iris_X = iris.data

# KMeans 演算法
kmeans_fit = cluster.KMeans(n_clusters = 4).fit(iris_X)
centers = kmeans_fit.cluster_centers_
# 印出分群結果
cluster_labels = kmeans_fit.labels_
print("分群結果：")
print(cluster_labels)
print("---")

# 印出品種看看
iris_y = iris.target
print("真實品種：")
print(iris_y)
print(centers)

In [ ]:
iris_X

In [ ]:

from pandas._testing import assert_frame_equal
df3 = pd.DataFrame(overSMOTE[0],columns=data.columns,dtype=object)
df3.index += 1
df3

In [ ]:
from pandas._testing import assert_frame_equal
df1 = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})
df2 = pd.DataFrame({'a': [1, 2], 'b': [3.0, 4.0]})
a = assert_frame_equal(df1, df1)
print(a)

In [ ]:
# oversampling SMOTE + RCSMOTE  Random 選擇 一半一半
# 沒有作用
import random 
import math
classCount = classprocess(output)
randomSmote = []
tempRCSMOTE = []
tempSMOTE = []
BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
Range = range_value(3,finaldata,output,data)
P_max = range_value(3,finaldata,output,data)[0]
Range = range_value(3,finaldata,output,data)[2]
for i in range(len(classCount)):
    count = math.ceil(int(classCount[i][1])/2); # 要產生多少數據  無條件捨去
    randomSmote.append([random.randint(0, len(output)-1) for _ in range(count)])
print(randomSmote)


In [ ]:
# data.shape[1] # 行數    data.shape[0] 列數
os.chdir("/Users/emily/Desktop/Research/oversampling_python/data/");
#print(os.getcwd());

data = pd.read_excel("pima.xlsx")
data.Class= data.Class.str.replace("\n", "").str.strip()

data = data.iloc[:,1:]
data.shape[1]
l = data.shape[1]-1

output = np.array(data.iloc[:,l]);
finaldata = np.array(data.iloc[:,:l])
data


In [ ]:
counter = Counter(output)
counter

In [ ]:
tedata = data
le = preprocessing.LabelEncoder()
for i in range(tedata.shape[1]):
    tedata.iloc[:,i] = le.fit_transform(tedata.iloc[:,i]) 

In [ ]:
# 分佈
counter = Counter(output)
for k,v in counter.items():
	per = v / len(output) * 100
	print("class",k,"數量：",v,"percentage",'%.3f' %per,"%")
	#print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.bar(counter.keys(), counter.values())
pyplot.show()
ir = 500/268;
print("IR ",ir)

In [ ]:
# 找出 knn
def knn(n_nu,sample):
     # n_nu 自己設定多少鄰近個點
     # input sample 沒有包含 class label
        nnarray = []
        neighbors=NearestNeighbors(n_neighbors=n_nu+1).fit(sample)
        for i in range(len(sample)):
            temp = neighbors.kneighbors(sample[i].reshape(1,-1),return_distance=False)[0]
            temp = np.delete(temp,0)
            # 有 array 存放 各個點的鄰近點 
            nnarray.append(temp)
        return nnarray # 回傳相近的點 分別是在第幾個 
karray = knn(3,finaldata)


In [ ]:
# 找出大類
def find_maj(sample_class):
    counter = Counter(sample_class);
    maj = list(dict(counter.most_common(1)).keys())
    maj = "".join(maj)
    return  maj

find_maj(output)

In [ ]:
# 判斷該點為什麼類型的點 
# 此 index 是 0 開始 所以對照到 excel 的要
# inminority 是否是要求在小類中的 如果是則放 true
def check_point_type(n_nu,sample,sample_class,data,inminority):
    # 使用 find_maj(find_maj(sample_class)
    # 使用 knn 找到鄰近的幾個點 
    point_type = [] # 放個點是屬於什麼類型的 border safe noisy
    n = 0
    b = 0
    s = 0
    maj = find_maj(sample_class) # 大類
    point = knn(n_nu,sample) # 回傳所有點鄰近 n_nu 個點
    #maj = "".join(maj) # maj 原本是 list 轉成 str
    for index,i in enumerate(point):
        maj_nu =0;
        if(data.iloc[index,data.shape[1]-1] == maj and inminority == True): # 如果該點本身是大類則不計算
            continue; # 如果使用此 則會跟論文的 data 一樣
        for j in point[index]: # 每個點鄰近個點的 loop 例如第一個點是 [1,2,3,4,5] 則 loop 裡面的數值
            if(data.iloc[j,data.shape[1]-1] == maj ): 
                maj_nu = maj_nu + 1 # 計算鄰近點為大類的有多少個
        if(maj_nu == n_nu ):
            point_type.append("noisy");
            n = n+1
        elif(n_nu/2 < maj_nu and maj_nu < n_nu ):
            point_type.append("border");
            b = b+1
        else:
            point_type.append("safe");
            s = s+1
    print("noisy",n,"\nborder",b,"\nsafe",s,"\nall",n+b+s)
    return point_type;
point_type = check_point_type(3,finaldata,output,data,True) # 論文使用 knn 為 3


In [ ]:
# 資料集變為小類的 border 跟 safe 以及 大類 去除 小類且為 noisy 的
"""
find the maximum value that existed for that attribute among
the minority samples of the 𝐵𝑆 set, and also, find the minimum value that existed for
the given attribute among the majority samples set
"""
# 先是小類去掉 noisy 再來是 大類 
def split_BS_Majdata(point_type,sample,sample_class):
    BS_sample = []
    Maj_sample = []
    return_sample = []
    maj = find_maj(sample_class) # 大類
    #maj = "".join(maj)
    for i in range(len(sample_class)):
        #print(sample_class[i])
        
        if(sample_class[i] == maj):
            Maj_sample.append(sample[i])
            
        elif(sample_class[i] != maj and point_type[i] != "noisy"):
            BS_sample.append(sample[i])
    return_sample = np.array([ BS_sample ,Maj_sample ])
  
    return return_sample
point_type = check_point_type(3,finaldata,output,data,False)
split_BS_Majdata(point_type,finaldata,output)


In [ ]:
def index_split_BS_Majdata(point_type,sample,sample_class): # maj 以及 BS set 在原本 data 的 index
    BS_sample = []
    Maj_sample = []
    return_sample = []
    maj = find_maj(sample_class) # 大類
    #maj = "".join(maj)
    for i in range(len(sample_class)):
        #print(sample_class[i])
        
        if(sample_class[i] == maj):
            Maj_sample.append(i)
            
        elif(sample_class[i] != maj and point_type[i] != "noisy"):
            BS_sample.append(i)
    return_sample = np.array([ BS_sample ,Maj_sample ])
    
    return return_sample
point_type = check_point_type(3,finaldata,output,data,False)
index_split_BS_Majdata(point_type,finaldata,output) 

In [ ]:
cr = Counter(output)
cr

In [ ]:
def classprocess(output):
    c = Counter(output)
    datagap = []
    maj = find_maj(output)
    maj_num = dict(c)[find_maj(output)]
    for className, number in c.items(): 
        #print(className," ",number)
        print(number)
        temp = np.array([className,(maj_num - number)])
        datagap.append(temp)
    return datagap

classprocess(output)

In [ ]:
"""
The obtained ranges for
all features (attributes) are used to control the location of the new synthetic samples in
data space.

The set of these ranges is denoted by 𝑅𝑎𝑛𝑔𝑒 =
(𝑟𝑎𝑛𝑔𝑒1 𝑟𝑎𝑛𝑔𝑒2 𝑟𝑎𝑛𝑔𝑒3 … 𝑟𝑎𝑛𝑔𝑒𝑛𝑎𝑡𝑡𝑟) array where 𝑛𝑎𝑡𝑡𝑟 is the number of
attributes in the dataset

如何產生 Range array
1. we find the maximum value that existed for that attribute among
the minority samples of the 𝐵𝑆 set  𝑃 _𝑚𝑎𝑥 = (𝑝𝑚𝑎𝑥1 𝑝𝑚𝑎𝑥2
…𝑝𝑚𝑎𝑥𝑛𝑎𝑡𝑡𝑟) array

2. find the minimum value that existed for
the given attribute among the majority samples set  𝑁 _𝑚𝑖𝑛 = (𝑛𝑚𝑖𝑛1 𝑛𝑚𝑖𝑛2 …𝑛𝑚𝑖𝑛𝑛𝑎𝑡𝑡𝑟)array

3. Then, the desired 𝑅𝑎𝑛𝑔𝑒 vector is obtained as the average of 𝑁_𝑚𝑖𝑛 and
𝑃 _𝑚𝑎𝑥 arrays


"""
# dataframe[0] 行數  dataframe[1] 列數
# 該屬性中 在 bs set 的最大值 以及 大類中的最小值
# range 為兩個相加除以 2
# index_split_BS_Majdata 回傳 [0] BS set [1] 回傳 Maj set
def range_value(n_nu,sample,sample_class,data):
    point_type = check_point_type(n_nu,sample,sample_class,data,False)
    index = index_split_BS_Majdata(point_type,sample,sample_class)
    BS_max = []
    Maj_min = []
    all_value = []
    for i in range(len(sample[0])): # loop 屬性
        for j in range(2): # loop data
            if(j==0): # BS set 的 index
                BS_index = index[j]
                max_value = np.max(sample[BS_index][i]);
                BS_max.append(max_value)
            else:
                Maj_index = index[j]
                min_value = np.min(sample[Maj_index][i]);
                Maj_min.append(min_value)
    # temp = np.array([BS_max,Maj_min]) 
    range_value = [(BS_max[i] - Maj_min[i])/2 for i in range(len(BS_max))]
    all_value = np.array([BS_max,Maj_min,range_value])
    return all_value
range_value(3,finaldata,output,data)

In [ ]:
# b𝑛𝑢𝑚: Number of samples in 𝐵𝑜𝑟𝑑𝑒𝑟 屬於 border 的資料數
# compute its(border sample) k nearest neighbors in BS, and save them in BSnn 存放 
# compute its(border sample) k nearest neighbors in Maj set, and save them in Majnn
def getBSnn_Majnn(n_nu,sample,sample_class,data):     
    point_type = check_point_type(n_nu,sample,sample_class,data,False) # 得知所有 sample 中屬於 border 的  
    index = index_split_BS_Majdata(point_type,sample,sample_class) 
    k = knn(n_nu,sample)
    BSnn = []
    Majnn = []
    # 用 border sample 的 index 
    for i,element in enumerate(point_type):
        if(element == "border"):
            for j in range(2):
                for w in index[j]:
                    if(j == 0 and i == w ):
                        BSnn.append(k[w])
                    elif(j==1 and i == w):
                        Majnn.append(k[w])
        
    temp = np.array([BSnn,Majnn]);
    print(len(temp[0]+temp[1]))
    return temp
    
len(getBSnn_Majnn(3,finaldata,output,data)[0][0])


In [ ]:
# 生成資料

def Populate(r,BSnn,Majnn,Range,P_max,sample,classbelong):
    # r 代表要產生的資料的數量 論文定義該數量為增加小類至跟大類相同的數量 達成 50% (大類減小類的數量)
    Bl = len(BSnn)-1
    nn_nu = len(BSnn[0])-1
    Ml = len(Majnn)-1
    Synthetic = []
    while(r > 0):

        s1row = random.randint(0,Bl)
        s1col = random.randint(0,nn_nu)
        s1 = BSnn[s1row][s1col]
        s2row = random.randint(0,Ml)
        s2col = random.randint(0,nn_nu)
        s2 = Majnn[s2row][s2col]
        min_attr = []
        for i in range(len(sample[0])):
        
            if(sample[s1][i] < sample[s2][i]):
                min_attr.append(sample[s1][i])
            else:
                min_attr.append(sample[s2][i])
        diff = [(P_max[i] - min_attr[i]) for i in range(len(P_max))]
        #print("diff",diff)
        gap = random.uniform(0,0.5)
        var = [(diff[i] * gap ) for i in range(len(diff))]
        temp = []
        for i in range(len(sample[0])):
            if(min_attr[i]+var[i] <= Range[i]):
                temp.append(min_attr[i]+var[i])
            else:
                temp.append(P_max[i]-var[i])
        temp.append(classbelong)
        Synthetic.append(temp)   
        r = r-1
    return Synthetic
    
needToGenerate = 232
BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
Range = range_value(3,finaldata,output,data)
P_max = range_value(3,finaldata,output,data)[0]
Range = range_value(3,finaldata,output,data)[2]
#Populate(needToGenerate,BSnn,Majnn,Range,P_max,finaldata)

In [ ]:
alldata

In [ ]:
# 每個類別都要 進行 populate 針對多類別
alldata = data.T
classCount = classprocess(output)
index = 1
for i in range(len(classCount)):
    #print(int(classCount[i][1]))
    if(int(classCount[i][1]) > 0):
        over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])
        length = alldata.shape[1]
        for j in range(len(over)):
            alldata[length+j] = over[j]
alldata

In [ ]:
X = alldata.T.iloc[:,:alldata.shape[0]]
y = alldata.T.iloc[:,alldata.shape[0]-1]
overc = Counter(y)
for i,v in overc.items():
    print("class",i,"number",v)

In [ ]:
# 一般 SMOTE
print("SMOTE")
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
modelD = DecisionTreeClassifier()
modelR = RandomForestClassifier(n_estimators=1000)
#clf = clf.fit(X, Y) # 
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoresRandomForest = cross_val_score(modelR, finaldata, output, scoring='roc_auc', cv=cv, n_jobs=-1)
scoresDecisionTree = cross_val_score(modelD, finaldata, output, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC for RandomForest: %.3f' % mean(scoresRandomForest))
print('Mean ROC AUC for DecisionTree: %.3f' % mean(scoresDecisionTree))

In [ ]:
scoresRandomForest

In [ ]:
# RCSMOTE
print("RCSMOTE")
X = alldata.T.iloc[:,:alldata.shape[0]-1]
y = alldata.T.iloc[:,alldata.shape[0]-1]
modelD = DecisionTreeClassifier()
modelR = RandomForestClassifier(n_estimators=1000)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoresRandomForest = cross_val_score(modelR, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
scoresDecisionTree = cross_val_score(modelD, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC for RandomForest: %.3f' % mean(scoresRandomForest))
print('Mean ROC AUC for DecisionTree: %.3f' % mean(scoresDecisionTree))

In [ ]:
# Borderline
print("Borderline")
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import BorderlineSMOTE
stepR = [('border_line', BorderlineSMOTE(random_state=42,kind="borderline-1")), ('model', RandomForestClassifier(n_estimators=1000))] # RandomForestClassifier(n_estimators=1000)
stepD = [('border_line', BorderlineSMOTE(random_state=42,kind="borderline-1")), ('model', DecisionTreeClassifier())] 
pipelineD = Pipeline(steps=stepD)
pipelineR = Pipeline(steps=stepR)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoresDecisionTree = cross_val_score(pipelineD, finaldata, output, scoring='roc_auc', cv=cv, n_jobs=-1)
scoresRandomForest = cross_val_score(pipelineR, finaldata, output, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC for RandomForest: %.3f' % mean(scoresRandomForest))
print('Mean ROC AUC for DecisionTree: %.3f' % mean(scoresDecisionTree))